In [1]:
# Check that imports for the rest of the file work.
import tensorflow.compat.v1 as tf
!pip install tensorflow-gan
import tensorflow_gan as tfgan
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Allow matplotlib images to render immediately.
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)  # Disable noisy outputs.

In [2]:
SP500_average = pd.read_csv(
    "./data_stock/SP500_average.csv",
    ).drop([0]).drop(columns=['Date'])
SP500_average.head()


,Open,High,Low,Close
1,2425.66,2553.93,2367.04,2529.19
2,2436.50,2453.57,2280.52,2398.10
3,2393.48,2466.97,2319.78,2409.39
4,2431.94,2453.01,2295.56,2304.92
5,2290.71,2300.73,2191.86,2237.40


In [3]:
def input_fn(mode, params):
  assert 'batch_size' in params
  assert 'noise_dims' in params
  bs = params['batch_size']
  nd = params['noise_dims']
  split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
  shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
  just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
  noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
              .map(lambda _: tf.random.normal([bs, nd])))
  if just_noise:
    return noise_ds
  SP500_average = pd.read_csv(
    "./data_stock/SP500_average.csv",
    ).drop([0]).drop(columns=['Date'])
  SP500_average_ds = tf.data.Dataset.from_tensor_slices(SP500_average.values.tolist()).cache()
  train_size = tf.data.experimental.cardinality(SP500_average_ds)
    
  if split == 'train':
    SP500_average_ds.take(train_size)
  else:
    SP500_average_ds.skip(train_size)
    
  if shuffle:
    SP500_average_ds = SP500_average_ds.shuffle(
        buffer_size=10000, reshuffle_each_iteration=True)
  SP500_average_ds = (SP500_average_ds.batch(bs, drop_remainder=True)
               .prefetch(tf.data.experimental.AUTOTUNE))

  return tf.data.Dataset.zip((noise_ds, SP500_average_ds))

In [4]:
# sanity check dataset
params = {'batch_size': 100, 'noise_dims':64}
with tf.Graph().as_default():
  ds = input_fn(tf.estimator.ModeKeys.TRAIN, params)
  example = next(iter(tfds.as_numpy(ds)))[1]
  print(example)

[[3814.98 3823.6  3792.86 3795.54]
 [3937.6  3949.13 3901.57 3910.52]
 [3166.44 3184.15 3142.93 3145.32]
 [4201.94 4213.38 4197.78 4200.88]
 [2908.83 2932.16 2902.88 2929.8 ]
 [3733.27 3760.2  3726.88 3756.07]
 [3969.31 3981.83 3943.25 3971.09]
 [4669.14 4714.13 4638.27 4713.07]
 [2738.65 2756.89 2657.67 2659.41]
 [4435.79 4445.21 4430.03 4436.75]
 [4256.97 4271.28 4256.97 4266.49]
 [4788.64 4804.06 4778.08 4793.06]
 [3892.59 3915.77 3892.59 3915.59]
 [4410.56 4444.35 4406.8  4441.67]
 [4392.74 4423.79 4373.   4423.15]
 [3543.76 3588.11 3535.23 3580.84]
 [4329.38 4371.6  4329.38 4369.55]
 [4642.99 4660.47 4606.52 4634.09]
 [3418.09 3432.09 3413.13 3431.28]
 [3840.27 3847.51 3816.68 3830.17]
 [2457.77 2571.42 2407.53 2475.56]
 [3694.73 3697.41 3678.88 3691.96]
 [3915.8  3925.02 3814.04 3829.34]
 [3406.46 3486.25 3405.17 3443.44]
 [4406.86 4422.18 4384.81 4387.16]
 [4474.81 4492.99 4445.7  4468.73]
 [4385.44 4415.88 4360.59 4361.19]
 [3441.42 3441.42 3364.86 3400.97]
 [3152.47 3186.82 31

In [5]:
def _dense(inputs, units, l2_weight, activation = None):
  return tf.layers.dense(
      inputs, units, activation,
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))


def _batch_norm(inputs, is_training):
  return tf.layers.batch_normalization(
      inputs, momentum=0.999, epsilon=0.001, training=is_training)

def _deconv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d_transpose(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=tf.nn.relu, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _conv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=None, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))


In [6]:
_leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)
def unconditional_generator(noise, mode, weight_decay=2.5e-5):
  """Generator to produce unconditional MNIST images."""
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  net = _dense(noise, 100, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 72, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 10, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 4, weight_decay)

  return net

  return net

In [7]:
_leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)

def unconditional_discriminator(price, unused_conditioning, mode, weight_decay=2.5e-5):
  del unused_conditioning
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  net = _dense(price, 100, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 72, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 10, weight_decay)
  net = _leaky_relu(net)
  
  net = _dense(net, 1, weight_decay)

  return net

In [8]:
from tensorflow_gan.examples.mnist import util as eval_util
import os

def get_eval_metric_ops_fn(gan_model):
  real_data_logits = tf.reduce_mean(gan_model.discriminator_real_outputs)
  gen_data_logits = tf.reduce_mean(gan_model.discriminator_gen_outputs)
  return {
      'real_data_logits': tf.metrics.mean(real_data_logits),
      'gen_data_logits': tf.metrics.mean(gen_data_logits),
  }

In [9]:
train_batch_size = 32 #@param
noise_dimensions = 4 #@param
generator_lr = 0.001 #@param
discriminator_lr = 0.0002 #@param

def gen_opt():
  gstep = tf.train.get_or_create_global_step()
  base_lr = generator_lr
  # Halve the learning rate at 1000 steps.
  lr = tf.cond(gstep < 1000, lambda: base_lr, lambda: base_lr / 2.0)
  return tf.train.AdamOptimizer(lr, 0.5)

gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=unconditional_generator,
    discriminator_fn=unconditional_discriminator,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    params={'batch_size': train_batch_size, 'noise_dims': noise_dimensions},
    generator_optimizer=gen_opt,
    discriminator_optimizer=tf.train.AdamOptimizer(discriminator_lr, 0.5),
    get_eval_metric_ops_fn=get_eval_metric_ops_fn)

In [10]:
# Disable noisy output.
tf.autograph.set_verbosity(0, False)

import time
steps_per_eval = 500 #@param
max_train_steps = 5000 #@param
batches_for_eval_metrics = 100 #@param

# Used to track metrics.
steps = []
real_logits, fake_logits = [], []
real_mnist_scores, mnist_scores, frechet_distances = [], [], []

cur_step = 0
start_time = time.time()
while cur_step < max_train_steps:
  
    
    
  next_step = min(cur_step + steps_per_eval, max_train_steps)

  start = time.time()
  gan_estimator.train(input_fn, max_steps=next_step)
  steps_taken = next_step - cur_step
  time_taken = time.time() - start
  print('Time since start: %.2f min' % ((time.time() - start_time) / 60.0))
  print('Trained from step %i to %i in %.2f steps / sec' % (
      cur_step, next_step, steps_taken / time_taken))
  cur_step = next_step
  
  # Calculate some metrics.
  metrics = gan_estimator.evaluate(input_fn, steps=batches_for_eval_metrics)
  steps.append(cur_step)
  real_logits.append(metrics['real_data_logits'])
  fake_logits.append(metrics['gen_data_logits'])
  print('Average discriminator output on Real: %.2f  Fake: %.2f' % (
      real_logits[-1], fake_logits[-1]))
  
  # Print current predictions
  iterator = gan_estimator.predict(
      input_fn, hooks=[tf.train.StopAtStepHook(num_steps=21)])
  try:
    daily_prices = np.array([next(iterator) for _ in range(20)])
  except StopIteration:
    pass
  for p in daily_prices:
    print(p)
 
  
  

C:\Users\zheng\AppData\Local\Temp/ipykernel_28652/1206689090.py:2: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(
C:\Users\zheng\anaconda3\lib\site-packages\keras\legacy_tf_layers\core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)


Time since start: 0.03 min
Trained from step 0 to 500 in 260.42 steps / sec
Average discriminator output on Real: 826.92  Fake: 0.07
[-0.6839396  -0.5093892  -0.029005   -0.35985088]
[-0.7331737  -0.46559614 -0.04137171 -0.4668968 ]
[-0.5983864  -0.49118713  0.01635591 -0.36307248]
[-0.39220762 -0.12818916  0.07325451 -0.18351325]
[-0.6725071  -0.39149603  0.097769   -0.42888764]
[-0.28325713 -0.15181573  0.0510788  -0.10019849]
[-0.46315226 -0.21304107  0.01682877 -0.23406969]
[-0.9201392  -0.8738374   0.02346537 -0.7861449 ]
[-0.31297907 -0.18801922  0.03637781 -0.12036449]
[-0.54106057 -0.45148578  0.05616545 -0.44779098]
[-0.20076112 -0.27815196 -0.0795952  -0.21969707]
[-1.1066921  -0.5715309   0.05576683 -0.6314213 ]
[-0.35729223 -0.19659129  0.03565373 -0.23032482]
[-0.23239015 -0.06281055  0.12312994 -0.11706465]
[-0.4450921  -0.32321656  0.01034956 -0.36300284]
[-0.4449376  -0.23843798  0.1071898  -0.31541196]
[-0.22992252 -0.10171118  0.01688176 -0.09088056]
[-0.5326958  -0.3